In [122]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import spacy
nlp = spacy.load('en_core_web_md')
import scipy.io
mat = scipy.io.loadmat('pascal50S.mat')


In [123]:
image_data = np.load('image_data_new.npy')

## final model

In [124]:
# categories = { 1: 'background', 2: 'aeroplane', 3: 'bicycle', 4: 'bird', 5: 'boat', 6: 'bottle', 7: 'bus', 8: 'car', 
#               9: 'cat', 10: 'chair', 11: 'cow', 12: 'dining table', 13: 'dog', 14: 'horse', 15: 'motorbike', 
#               16: 'person', 17: 'potted plant', 18: 'sheep', 19: 'sofa', 20: 'train',21:'tv/monitor'}
# pre_trained = cv2.dnn.readNetFromCaffe('MobileNetSSD_deploy.prototxt.txt','MobileNetSSD_deploy.caffemodel')
class_names = ['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','dining table','dog','horse','motorbike','person','potted plant','sheep','sofa','train','tv/monitor']
print(len(class_names))

20


In [125]:


input_data = mat['train_sent_final']
# input_data
label_desc_temp=[]
label_desc=[]
for i in range(1000):
  for j in range(50):
    label_desc_temp.append(input_data[0][i][1][0][j][0])
  label_desc.append(label_desc_temp)
  label_desc_temp=[]
    
# true_annotation=['blank','aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','dining table','dog','horse','motorbike','person','potted plant','sheep','sofa','train','tv/monitor']

In [113]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english') 
distinct_word=[]
final_list=[]
symbols="./?><;:{}[]!@#$%^&*()_+=-~`',/*-+123456789"
for everylist in label_desc:
  for everyline in everylist:
    words=everyline.split()

    #########cleaning
    for word in  words:
      for alpha in word:
        if alpha in symbols:
          word = word.replace(alpha,'')
      word=word.lower()
      ########
    
      if word not in distinct_word and word not in stop_words:
        distinct_word.append(word)
      # if word not in stop_words:
        # distinct_word.append(stemmer.stem(word))
  final_list.append(distinct_word)
  distinct_word=[]


In [114]:
# final_list[250]

In [115]:
word_list=[]
word_list_final=[]
import operator
for i in range(len(final_list)):
  dicto={}
  for item in final_list[i]:
    if item in dicto.keys():
      dicto[item]+=1
    else:
      dicto[item]=1

  sorted_x = sorted(dicto.items(), key=operator.itemgetter(1),reverse=True)

  for j in range(10):
    word_list.append(sorted_x[j][0])
  
  word_list_final.append(word_list)
  word_list=[]


In [116]:
word_list_final

[['man',
  'boat',
  'fishing',
  'canoe',
  'lone',
  'fisherman',
  'rowboat',
  'empty',
  'lake',
  'person'],
 ['train',
  'flags',
  'snow',
  'two',
  'flying',
  'engine',
  'passes',
  'fly',
  'cloudy',
  'landscape'],
 ['car',
  'port',
  'appears',
  'deserted',
  'signs',
  'posted',
  'different',
  'routes',
  'bus',
  'depot'],
 ['team',
  'planes',
  'stream',
  'past',
  'group',
  'fly',
  'formation',
  'contrails',
  'following',
  'flies'],
 ['group',
  'military',
  'couples',
  'enjoying',
  'nice',
  'dinner',
  'three',
  'men',
  'women',
  'eating'],
 ['man',
  'kneels',
  'pavement',
  'cell',
  'phone',
  'taking',
  'notes',
  'kneeling',
  'sidewalk',
  'order'],
 ['three',
  'bottles',
  'shelf',
  'empty',
  'desk',
  'bottle',
  'aftershave',
  'men',
  'hygiene',
  'product'],
 ['yawning',
  'dog',
  'lying',
  'deck',
  'porch',
  'barking',
  'sits',
  'front',
  'lies',
  'wooden'],
 ['tealoufitted',
  'rider',
  'races',
  'grey',
  'horse',
  'j

## from here images cnn

In [117]:
# categories = { 1: 'background', 2: 'aeroplane', 3: 'bicycle', 4: 'bird', 5: 'boat', 6: 'bottle', 7: 'bus', 8: 'car', 
#               9: 'cat', 10: 'chair', 11: 'cow', 12: 'dining table', 13: 'dog', 14: 'horse', 15: 'motorbike', 
#               16: 'person', 17: 'potted plant', 18: 'sheep', 19: 'sofa', 20: 'train',21:'tv/monitor'}

pre_trained = cv2.dnn.readNetFromCaffe('MobileNetSSD_deploy.prototxt.txt','MobileNetSSD_deploy.caffemodel')

class_names = ['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','dining table','dog','horse','motorbike','person','potted plant','sheep','sofa','train','tv/monitor']
# print(len(class_names))

In [118]:

# image = image_data[12]
# fig = plt.figure(figsize=(8,8))
# plt.imshow(image)
# plt.title('input image')
# plt.xticks([])
# plt.yticks([])

In [119]:
images_labels = []
# labelling = []
for x in range(len(final_list)):
    labelling = []
    image = image_data[x]
    image = cv2.cvtColor(image,cv2.COLOR_BAYER_BG)
    img_resize = cv2.resize(image,(300,300))
    blob = cv2.dnn.blobFromImage(img_resize,0.007843,(300,300),127.5)
    pre_trained.setInput(blob)
    detections = pre_trained.forward()
    colors = np.random.uniform(255,0,size = (len(class_names),3))
    (h, w) = img_resize.shape[:2]
    for i in np.arange(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence >0.:
            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
    #         label = "{}: {:.2f}%".format(categories[idx], confidence * 100)
            label = "{}: {:.2f}%".format(class_names[idx-1], confidence * 100)
            labelling.append(class_names[idx-1])
            labelling = list(set(labelling))
            

    images_labels.append(labelling)


error: OpenCV(4.5.1) /tmp/pip-req-build-jr1ur_cf/opencv/modules/dnn/src/layers/convolution_layer.cpp:379: error: (-2:Unspecified error) Number of input channels should be multiple of 3 but got 4 in function 'getMemoryShapes'


In [120]:
images_labels

[]

## Final

In [88]:
temp_zipped = [list(t) for t in zip(word_list_final,images_labels)]

In [89]:
# np.array(temp_zipped).shape

In [90]:
final = []
for x in range(len(temp_zipped)):
    final.append(temp_zipped[x][0] + temp_zipped[x][1])

In [91]:
final

[['man',
  'boat',
  'fishing',
  'canoe',
  'lone',
  'fisherman',
  'rowboat',
  'empty',
  'lake',
  'person',
  'aeroplane'],
 ['train',
  'flags',
  'snow',
  'two',
  'flying',
  'engine',
  'passes',
  'fly',
  'cloudy',
  'landscape',
  'train',
  'aeroplane',
  'boat'],
 ['car',
  'port',
  'appears',
  'deserted',
  'signs',
  'posted',
  'different',
  'routes',
  'bus',
  'depot',
  'chair'],
 ['team',
  'planes',
  'stream',
  'past',
  'group',
  'fly',
  'formation',
  'contrails',
  'following',
  'flies',
  'aeroplane'],
 ['group',
  'military',
  'couples',
  'enjoying',
  'nice',
  'dinner',
  'three',
  'men',
  'women',
  'eating',
  'bottle',
  'dining table',
  'person'],
 ['man',
  'kneels',
  'pavement',
  'cell',
  'phone',
  'taking',
  'notes',
  'kneeling',
  'sidewalk',
  'order',
  'car',
  'person',
  'bicycle'],
 ['three',
  'bottles',
  'shelf',
  'empty',
  'desk',
  'bottle',
  'aftershave',
  'men',
  'hygiene',
  'product',
  'bottle'],
 ['yawning'

In [92]:
final_labels_list = []
count = 0
for images in range(len(final)):
    label=[0 for i in range(len(class_names))]
#     print(len(label))
    for index, item in enumerate(class_names):
        name = nlp(item)
        for item1 in final[images]:
            name1 = nlp(item1)
            if name.similarity(name1)>0.69: 
                label[index]=1
                break
    count += 1
    print(count)
    if 'tv' in final_list[images] or 'monitor' in final_list[images]:
        label[19] = 1
    final_labels_list.append(label)
    
    

1
2
3
4
5
6
7
8


<ipython-input-92-b3d6c2907280>:10: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if name.similarity(name1)>0.69:


9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281


In [100]:
for index,items in enumerate(final_labels_list):
    j=0
    for i in items:
        if j==19 and i==1:
            print(index)
        j+=1

49
69
75
97
109
125
148
157
170
189
221
224
252
253
266
270
305
308
317
338
340
344
362
368
370
379
398
421
425
441
442
447
455
467
468
476
490
513
544
548
574
593
627
665
685
688
697
742
747
764
767
775
778
782
784
795
802
807
819
835
864
869
896
925
931
942
948
969
990


In [121]:
import pickle
# l = [1,2,3,4]
with open("final_labels.txt", "wb") as fp:   #Pickling
    pickle.dump(final_labels_list, fp)

In [ ]:
# final[2]

# images with texts

In [ ]:
# images_labels = []

# def multi_label_classification(x):
#     labelling = []
# # for x in range(len(final_list)):
# #     labelling = []
#     image = image_data[x]
#     img_resize = cv2.resize(image,(300,300))
#     blob = cv2.dnn.blobFromImage(img_resize,0.007843,(300,300),127.5)
#     pre_trained.setInput(blob)
#     detections = pre_trained.forward()
#     colors = np.random.uniform(255,0,size = (len(class_names),3))
#     (h, w) = img_resize.shape[:2]
#     for i in np.arange(0, detections.shape[2]):
#         confidence = detections[0, 0, i, 2]
#         if confidence >0.:
#             idx = int(detections[0, 0, i, 1])
#             box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
#             (startX, startY, endX, endY) = box.astype("int")
#     #         label = "{}: {:.2f}%".format(categories[idx], confidence * 100)
#             label = "{}: {:.2f}%".format(class_names[idx-1], confidence * 100)
#             labelling.append(class_names[idx-1])
#             labelling = list(set(labelling))
            
#             cv2.rectangle(img_resize, (startX, startY), (endX, endY), colors[idx], 2)     
#             y = startY - 15 if startY - 15>15 else startY + 15     
#             cv2.putText(img_resize, label, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[idx], 2)
            
# #     images_labels.append(labelling)
#     cv2.imshow("Output", img_resize)
#     cv2.waitKey(4000)
#     cv2.destroyAllWindows()
#     fig = plt.figure()
#     ax1 = fig.add_axes((0.1, 0.2, 0.8, 0.7))
#     # ax1.set_title('object detection')
#     plt.axis("off")
#     plt.imshow(img_resize)
#     plt.show()
# #     print(list(set(labelling)))
#     print(f'The labels from image classification for given image is :{labelling}')
#     return

In [107]:
# multi_label_classification(224)

IndexError: index 20 is out of bounds for axis 0 with size 20